In [354]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from scipy import fft
import cv2
from matplotlib.pylab import mpl



In [355]:
#   数据预处理
path_A='raw_data/Accelerometer.csv'
path_G='raw_data/Gyroscope.csv'
path_M='raw_data/Magnetometer.csv'
path_L='raw_data/Location.csv'
#   读取原始数据
Data_A=pd.read_csv(path_A)
Data_G=pd.read_csv(path_G)
Data_M=pd.read_csv(path_M)
Data_L=pd.read_csv(path_L)


In [356]:
#   将加速度和角速度数据按时间对齐
Data_A=np.array(Data_A.iloc[3:,1:])
Data_G=np.array(Data_G.iloc[:-1,1:])
Time=np.arange(0.02,216.05,0.01)


In [357]:

def normalization(data):
    _range = np.max(data) - np.min(data)
    return (data - np.min(data)) / _range


In [358]:
def fftPlot(data):
    y=data
    N=len(data)
    n=np.arange(N)
    half_n=n[int(N/2)]
    fft_y=fft.fft(y)
    abs_y=np.abs(fft_y)
    angle_y=np.angle(fft_y)
    normalization_y=normalization(abs_y)
    normalization_half_y=np.abs(fft.rfft(data))

#------------------plot--------------------------
    sns.set()
    plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']   #显示中文
    mpl.rcParams['axes.unicode_minus']=False       #显示负号
    fs=5    #fontsize
    plt.figure(dpi=300)
    plt.subplot(231)
    plt.plot(n,y)
    plt.title('原始波形',fontsize=fs)

    plt.subplot(232)
    plt.plot(n,fft_y,'black')
    plt.title('双边振幅谱(未求振幅绝对值)',fontsize=fs)

    plt.subplot(233)
    plt.plot(n,abs_y,'r')
    plt.title('双边振幅谱(未归一化)',fontsize=fs)

    plt.subplot(234)
    plt.plot(n,angle_y,'violet')
    plt.title('双边相位谱(未归一化)',fontsize=fs)

    plt.subplot(235)
    plt.plot(n,normalization_y,'g')
    plt.title('双边振幅谱(归一化)',fontsize=fs)

    plt.subplot(236)
    plt.plot(np.arange(len(normalization_half_y)),normalization_half_y,'blue')
    plt.title('单边振幅谱(归一化)',fontsize=fs)
    plt.xlim((0,2500))
    plt.ylim((0,1000))
    plt.tight_layout()
    plt.show()

In [359]:
#   利用FFT降噪
def nc_fft(data, step, threshold, plot=False):
    n=len(data)
    y=fft.rfft(data)
    x=fft.rfftfreq(n,step)
    y_abs=np.abs(y)
    indices=y_abs>threshold
    y_clean=indices*y
    data_clean=fft.irfft(y_clean)
    if plot:
        plt.figure(dpi=500)
        plt.plot(np.arange(len(data)),data,color='red',label='origin')
        plt.plot(np.arange(len(data_clean)),data_clean,color='blue',label='nc')
        plt.legend()
        plt.show()
    return data_clean


In [360]:
#   将加速度的每个分量进行FFT降噪
step=0.01
threshold=5000
a_x=Data_A[:,0]
a_y=Data_A[:,1]
a_z=Data_A[:,2]


a_x_c=nc_fft(a_x,step,threshold)
a_y_c=nc_fft(a_y,step,threshold)
a_z_c=nc_fft(a_z,step,threshold)

a_c=np.concatenate((a_x_c.reshape((-1,1)),a_y_c.reshape((-1,1)),a_z_c.reshape((-1,1))),axis=1)

In [365]:
#   对角速度进行FFT降噪
threshold=500
g_x=Data_G[:,0]
g_y=Data_G[:,1]
g_z=Data_G[:,2]
# fftPlot(g_x)
# fftPlot(g_y)
# fftPlot(g_z)
g_x_c=nc_fft(g_x,step,threshold)
g_y_c=nc_fft(g_y,step,threshold)
g_z_c=nc_fft(g_z,step,threshold)

g_c=np.concatenate((g_x_c.reshape((-1,1)),g_y_c.reshape((-1,1)),g_z_c.reshape((-1,1))),axis=1)